In [4]:
from __future__ import division, print_function
import glob
import os
import cv2
import PIL
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
import torch.nn.init
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import torchvision.datasets as dset
import torchvision.transforms as transforms
from tqdm import tqdm
from torch.autograd import Variable
from copy import deepcopy, copy
from scipy.optimize import linear_sum_assignment
import sys
import matplotlib.pyplot as plt
import random as rand
from scipy import linalg

In [38]:
p11 = [[81,123], [594,409], [848,504], [167,212], [983,300], [146,581], [57,482],   [242,494], [213,56], [944,639] ]
p12 = [[126,131],[609,412], [857,508], [200,216], [1017,302],[177,579], [97,480],   [268,494], [251,63], [966,647] ]
p21 = [[95,303], [309,433], [36,176],  [419,192], [623,451], [575,224], [157,176],  [73,536],  [398,75], [229,145] ]
p22 = [[118,302],[318,433], [70,175],  [451,191], [639,450], [605,223], [192,175],  [72,536],  [418,74], [265,144] ]
p31 = [[135,50], [52,253],  [165,164], [170,28],  [289,78],  [253,293], [386,112],  [418,11],  [223,197],[418,211] ]
p32 = [[100,49], [58,253],  [159,162], [135,27],  [255,78],  [236,293], [351,112],  [383,12],  [215,196],[383,211] ]
p41 = [[22,152], [109,149], [87,340],  [438,498], [259,504], [448,679], [355,619],  [479,145], [259,153],[196,358] ]
p42 = [[10,163], [108,139], [83,349],  [515,530], [301,537], [539,756], [468,657],  [521,149], [282,138],[217,363] ]
p51 = [[96,236], [366,36],  [314,554], [105,1087],[531,1118],[978,798], [1180,1028],[1192,482],[818,129],[1118,240]]
p52 = [[170,239],[441,41],  [388,557], [172,1089],[601,1118],[1052,800],[1254,1030],[1266,485],[893,133],[1193,244]]

In [59]:
choose = 5  # select between 5 pairs of images (1~5)

if choose == 1:
    path_p = 'Stereo_images/image11.png'
    path_q = 'Stereo_images/image12.png'
    p = p11
    q = p12
    save = '1_'
    txtName1 = 't1_1.txt'
    txtName2 = 't1_2.txt'
elif choose == 2:
    path_p = 'Stereo_images/image21.png'
    path_q = 'Stereo_images/image22.png'    
    p = p21
    q = p22
    save = '2_'
    txtName1 = 't2_1.txt'
    txtName2 = 't2_2.txt'
elif choose == 3:
    path_p = 'Stereo_images/image31.png'
    path_q = 'Stereo_images/image32.png'    
    p = p31
    q = p32
    save = '3_'
    txtName1 = 't3_1.txt'
    txtName2 = 't3_2.txt'
elif choose == 4:
    path_p = 'Stereo_images/image41.png'
    path_q = 'Stereo_images/image42.png'    
    p = p41
    q = p42
    save = '4_'
    txtName1 = 't4_1.txt'
    txtName2 = 't4_2.txt'
elif choose == 5:
    path_p = 'Stereo_images/image51.png'
    path_q = 'Stereo_images/image52.png'    
    p = p51
    q = p52
    save = '5_'
    txtName1 = 't5_1.txt'
    txtName2 = 't5_2.txt'

In [60]:
p_point = torch.load("p_keypoints.pt")
q_point = torch.load("q_keypoints.pt")
print("p_keypoints: ", p_point.shape)
print("q_keypoints: ", q_point.shape)

def plotDesc(point, path, color):
    x_list = []
    y_list = []
    for i in range(len(point[choose-1])):
        x_list.append(point[choose-1][i][0])
        y_list.append(point[choose-1][i][1])

    im = plt.imread(path)
    implot = plt.imshow(im)
    plt.plot(x_list, y_list, color)
    plt.show()    

# # plot image 1 
# plotDesc(p_point, path_p, 'b.') 
# # plot image 2
# plotDesc(q_point, path_q, 'r.') 

p_keypoints:  torch.Size([5, 100, 2])
q_keypoints:  torch.Size([5, 100, 2])


In [61]:
p_des = torch.load("p_description.pt")  # torch.Size([5, 100, 128])
q_des = torch.load("q_description.pt")  # torch.Size([5, 100, 128])
p_point = np.array(p_point)
q_point = np.array(q_point)

data = np.zeros((100, 100))
pair_matrix = data.tolist()
cost_matrix = torch.tensor(data)

for i in range(len(p_des[choose-1])):       # 100
    for j in range(len(q_des[choose-1])):   # 100
        pair_matrix[i][j] = (p_point[choose-1][i].tolist(), q_point[choose-1][j].tolist())
        cost_matrix[i][j] = torch.dist(p_des[choose-1][i], q_des[choose-1][j], p=2)

cost_matrix_np = cost_matrix.detach().numpy()
row_ind, col_ind = linear_sum_assignment(cost_matrix_np)   # Hungarian

# C: one-to-one matching pairs
c = []
for i in range(len(col_ind)):
    c.append(pair_matrix[i][col_ind[i]])
    c[i][0].append(1)
    c[i][1].append(1)
# print("\nC:\n", c)    
print("shape of C:", np.shape(c))    # (100, 2, 3)

shape of C: (100, 2, 3)


In [62]:
def plotFromC(c, idx, path, color):
    x_list = []
    y_list = []
    for i in range(len(c)):
        x_list.append(c[i][idx][0])
        y_list.append(c[i][idx][1])
    im = plt.imread(path)
    implot = plt.imshow(im)
    plt.plot(x_list, y_list, color)
    plt.show()

# # plot image 1 
# plotFromC(c, 0, path_p, 'b.')
# # plot image 2 
# plotFromC(c, 1, path_q, 'r.')

In [63]:
f = open(txtName1, "w")

for i, item in enumerate(c):
    f.write('{:>20}  {:>20}\n'.format(str(item[0][0]), str(item[0][1])))
    
f.close()

f = open(txtName2, "w")

for i, item in enumerate(c):
    f.write('{:>20}  {:>20}\n'.format(str(item[1][0]), str(item[1][1])))
    
f.close()